In [27]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

# Basic imports
import numpy as np
import xarray as xr
import xdatasets as xd
from scipy.stats import genextreme, genpareto, gumbel_r

import xhydro as xh
from xhydro.extreme_value_analysis.parameterestimation import *
import xclim.indices.stats as stats
import warnings
warnings.simplefilter("always", UserWarning)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
ds = (
    xd.Query(
        **{
            "datasets": {
                "deh": {
                    "id": ["020*"],
                    "regulated": ["Natural"],
                    "variables": ["streamflow"],
                }
            },
            "time": {"start": "1970-01-01", "minimum_duration": (15 * 365, "d")},
        }
    )
    .data.squeeze()
    .load()
)

# This dataset lacks some of the aforementioned attributes, so we need to add them.
ds["id"].attrs["cf_role"] = "timeseries_id"
ds["streamflow"].attrs = {
    "long_name": "Streamflow",
    "units": "m3 s-1",
    "standard_name": "water_volume_transport_in_river_channel",
    "cell_methods": "time: mean",
}

# Some examples
timeargs = {
    "spring": {"date_bounds": ["02-11", "06-19"]},
    "summer": {"doy_bounds": [152, 243]},
    "fall": {"month": [9, 10, 11]},
    "winter": {
        "season": ["DJF"],
        "freq": "YS-DEC",
    },  # To correctly wrap around the year, we need to specify the resampling frequency.
    "august": {"month": [8]},
    "annual": {},
}
# Here, we hide years with more than 15% of missing data.
ds_4fa = xh.indicators.get_yearly_op(
    ds, op="max", timeargs=timeargs, missing="pct", missing_options={"tolerance": 0.15}
)

# Create a mask beforehand
import random

nyears = np.unique(ds.time.dt.year).size
dom_start = xr.DataArray(
    np.random.randint(1, 30, size=(nyears,)).astype("str"),
    dims=("year"),
    coords={"year": np.unique(ds.time.dt.year)},
)
dom_end = xr.DataArray(
    np.random.randint(1, 30, size=(nyears,)).astype("str"),
    dims=("year"),
    coords={"year": np.unique(ds.time.dt.year)},
)

mask = xr.zeros_like(ds["streamflow"])
for y in dom_start.year.values:
    # Random mask of dates per year, between April and June.
    mask.loc[
        {
            "time": slice(
                str(y) + "-04-" + str(dom_start.sel(year=y).item()),
                str(y) + "-06-" + str(dom_end.sel(year=y).item()),
            )
        }
    ] = 1
# The name of the indexer will be used to identify the variable created here
timeargs_custom = {"custom": {}}

# We use where() to mask the data that we want to ignore
masked = ds.where(mask == 1)
# Since we masked almost all of the year, our tolerance for missing data should be changed accordingly
missing = "at_least_n"
missing_options = {"n": 45}

# We use xr.merge() to combine the results with the previous dataset.
ds_4fa = xr.merge(
    [
        ds_4fa,
        xh.indicators.get_yearly_op(
            masked,
            op="max",
            timeargs=timeargs_custom,
            missing=missing,
            missing_options=missing_options,
        ),
    ]
)
# Get a daily volume from a daily streamflow
# ds["volume"] = xh.indicators.compute_volume(ds["streamflow"], out_units="hm3")

# We'll take slightly different indexers
timeargs_vol = {"spring": {"date_bounds": ["04-30", "06-15"]}, "annual": {}}

# The operation that we want here is the sum, not the max.
# ds_4fa = xr.merge(
#     [
#         ds_4fa,
#         xh.indicators.get_yearly_op(
#             ds,
#             op="sum",
#             input_var="volume",
#             timeargs=timeargs_vol,
#             missing="pct",
#             missing_options={"tolerance": 0.15},
#             interpolate_na=True,
#         ),
#     ]
# )

ds_4fa = ds_4fa.assign_coords(temperature=("temperature", np.linspace(25, 35, 56)))
temperature = ds_4fa["temperature"].values.tolist()
ds_4fa

<xarray.Dataset> Size: 10kB
Dimensions:                (id: 5, time: 56, temperature: 56)
Coordinates: (12/16)
    drainage_area          (id) float32 20B 1.09e+03 647.0 59.8 626.0 1.2e+03
    end_date               (id) datetime64[ns] 40B 2006-10-13 ... 1996-08-13
  * id                     (id) object 40B '020302' '020404' ... '020802'
    latitude               (id) float32 20B 48.77 48.81 48.98 48.98 49.2
    longitude              (id) float32 20B -64.52 -64.92 -64.43 -64.7 -65.29
    name                   (id) object 40B 'Saint' 'York' ... 'Madeleine'
    ...                     ...
    start_date             (id) datetime64[ns] 40B 1989-08-12 ... 1970-01-01
    time_agg               <U4 16B 'mean'
    timestep               <U1 4B 'D'
    variable               <U10 40B 'streamflow'
  * time                   (time) datetime64[ns] 448B 1970-01-01 ... 2025-01-01
  * temperature            (temperature) float64 448B 25.0 25.18 ... 34.82 35.0
Data variables:
    streamflow_max_spring  (id, time) float32 1kB nan nan nan ... nan nan nan
    streamflow_max_summer  (id, time) float32 1kB nan nan nan ... nan nan nan
    streamflow_max_fall    (id, time) float32 1kB nan nan nan ... nan nan nan
    streamflow_max_august  (id, time) float32 1kB nan nan nan ... nan nan nan
    streamflow_max_annual  (id, time) float32 1kB nan nan nan ... nan nan nan
    streamflow_max_winter  (id, time) float32 1kB nan nan nan ... nan nan nan
    streamflow_max_custom  (id, time) float32 1kB nan nan nan ... nan nan nan
Attributes:
    cat:frequency:         yr
    cat:processing_level:  indicators
    cat:id:

In [15]:
type(ds_4fa.streamflow_max_spring.data)

numpy.ndarray

In [12]:
params = fit(ds_4fa, locationcov=["temperature"], logscalecov=["temperature"], dist="genextreme", method="ml", confidence_level=0.9)
params

gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
print debug error
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
print debug error
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called


┌ Warning: The maximum likelihood algorithm did not find a solution. Maybe try with different initial values or with another method. The returned values are the initial values.
└ @ Extremes C:\Users\HP\.julia\packages\Extremes\4Tbee\src\parameterestimation\maximumlikelihood.jl:20


gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
print debug error
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called
gevfit called


┌ Warning: The maximum likelihood algorithm did not find a solution. Maybe try with different initial values or with another method. The returned values are the initial values.
└ @ Extremes C:\Users\HP\.julia\packages\Extremes\4Tbee\src\parameterestimation\maximumlikelihood.jl:20


<xarray.Dataset> Size: 6kB
Dimensions:                      (id: 5, dparams: 5, temperature: 56)
Coordinates: (12/16)
    drainage_area                (id) float32 20B 1.09e+03 647.0 ... 1.2e+03
    end_date                     (id) datetime64[ns] 40B 2006-10-13 ... 1996-...
  * id                           (id) object 40B '020302' '020404' ... '020802'
    latitude                     (id) float32 20B 48.77 48.81 48.98 48.98 49.2
    longitude                    (id) float32 20B -64.52 -64.92 ... -64.7 -65.29
    name                         (id) object 40B 'Saint' 'York' ... 'Madeleine'
    ...                           ...
    start_date                   (id) datetime64[ns] 40B 1989-08-12 ... 1970-...
    time_agg                     <U4 16B 'mean'
    timestep                     <U1 4B 'D'
    variable                     <U10 40B 'streamflow'
  * temperature                  (temperature) float64 448B 25.0 25.18 ... 35.0
  * dparams                      (dparams) <U27 540B 'shape' ... 'scale_tempe...
Data variables: (12/21)
    streamflow_max_spring        (id, dparams) float64 200B nan nan ... 1.079
    streamflow_max_summer        (id, dparams) float64 200B 0.3693 ... 0.9357
    streamflow_max_fall          (id, dparams) float64 200B 0.125 ... 1.051
    streamflow_max_august        (id, dparams) float64 200B -0.7256 ... 0.9634
    streamflow_max_annual        (id, dparams) float64 200B nan nan ... 1.095
    streamflow_max_winter        (id, dparams) float64 200B nan nan ... 1.172
    ...                           ...
    streamflow_max_summer_upper  (id, dparams) float64 200B 1.135 ... 1.121
    streamflow_max_fall_upper    (id, dparams) float64 200B 0.6151 ... 1.282
    streamflow_max_august_upper  (id, dparams) float64 200B 0.5695 ... 1.264
    streamflow_max_annual_upper  (id, dparams) float64 200B nan nan ... 1.408
    streamflow_max_winter_upper  (id, dparams) float64 200B nan nan ... 2.725
    streamflow_max_custom_upper  (id, dparams) float64 200B 0.5294 ... 1.407
Attributes:
    cat:frequency:         yr
    cat:processing_level:  indicators
    cat:id:                
    long_name:             genextreme parameters
    description:           Parameters of the genextreme distribution
    method:                ML
    estimator:             Maximum likelihood
    scipy_dist:            genextreme
    units:                 
    history:               [2024-08-08 14:14:40] fit: Estimate distribution p...

┌ Warning: The maximum likelihood algorithm did not find a solution. Maybe try with different initial values or with another method. The returned values are the initial values.
└ @ Extremes C:\Users\HP\.julia\packages\Extremes\4Tbee\src\parameterestimation\maximumlikelihood.jl:20


# Non stationary

In [21]:
from xarray import open_zarr
import glob
import os

# path_hist = glob.glob(os.path.join(r"C:\Users\KAMIL PC\Desktop\Life\Stage\Documents\non_stationary_data\eau_precpitable_CanESM5"))
path_hist = glob.glob(os.path.join(r"C:\Users\HP\Desktop\Stage\non_stationary_data\kamil_non_stationary_data"))

ds_ns = open_zarr(path_hist[0], consolidated=True)
ds_ns = ds_ns.isel(Station=slice(0, 5))
ds_ns = ds_ns.compute()
ds_ns

<xarray.Dataset> Size: 6kB
Dimensions:      (Station: 5, time: 65)
Coordinates:
  * Station      (Station) object 40B '010101' '010802' ... '011003' '011201'
    source       <U18 72B 'era5_single_levels'
    spatial_agg  <U7 28B 'polygon'
  * time         (time) datetime64[ns] 520B 1960-12-31 1961-12-31 ... 2024-12-31
    timestep     <U1 4B 'A'
Data variables:
    t2m_nanmean  (Station, time) float64 3kB 276.1 275.6 274.7 ... 277.0 264.8
    tp_nansum    (Station, time) float64 3kB 1.14 1.004 ... 1.181 0.0001042
Attributes: (12/30)
    GRIB_NV:                                  0
    GRIB_Nx:                                  177
    GRIB_Ny:                                  93
    GRIB_cfName:                              unknown
    GRIB_cfVarName:                           tp
    GRIB_dataType:                            fc
    ...                                       ...
    GRIB_totalNumber:                         0
    GRIB_typeOfLevel:                         surface
    GRIB_units:                               m
    long_name:                                Total precipitation
    standard_name:                            unknown
    units:                                    m

In [28]:
params_ns = fit(ds_ns, dist="genextreme", method="ml")
params_ns

C:\Users\HP\Desktop\Stage\xhydro\src\xhydro\extreme_value_analysis\parameterestimation.py:73: UserWarning: There was an error in fitting the data to a genextreme distribution. Returned parameters are numpy.nan.
C:\Users\HP\Desktop\Stage\xhydro\src\xhydro\extreme_value_analysis\parameterestimation.py:73: UserWarning: There was an error in fitting the data to a genextreme distribution. Returned parameters are numpy.nan.
C:\Users\HP\Desktop\Stage\xhydro\src\xhydro\extreme_value_analysis\parameterestimation.py:73: UserWarning: There was an error in fitting the data to a genextreme distribution. Returned parameters are numpy.nan.
C:\Users\HP\Desktop\Stage\xhydro\src\xhydro\extreme_value_analysis\parameterestimation.py:73: UserWarning: There was an error in fitting the data to a genextreme distribution. Returned parameters are numpy.nan.
C:\Users\HP\Desktop\Stage\xhydro\src\xhydro\extreme_value_analysis\parameterestimation.py:73: UserWarning: There was an error in fitting the data to a genex

<xarray.Dataset> Size: 924B
Dimensions:            (Station: 5, dparams: 3)
Coordinates:
  * Station            (Station) object 40B '010101' '010802' ... '011201'
    source             <U18 72B 'era5_single_levels'
    spatial_agg        <U7 28B 'polygon'
    timestep           <U1 4B 'A'
  * dparams            (dparams) <U5 60B 'shape' 'loc' 'scale'
Data variables:
    t2m_nanmean        (Station, dparams) float64 120B nan nan nan ... nan nan
    tp_nansum          (Station, dparams) float64 120B -0.4625 1.059 ... 0.2151
    t2m_nanmean_lower  (Station, dparams) float64 120B nan nan nan ... nan nan
    tp_nansum_lower    (Station, dparams) float64 120B -0.5505 1.002 ... 0.1753
    t2m_nanmean_upper  (Station, dparams) float64 120B nan nan nan ... nan nan
    tp_nansum_upper    (Station, dparams) float64 120B -0.3744 1.117 ... 0.2639
Attributes: (12/37)
    GRIB_NV:                                  0
    GRIB_Nx:                                  177
    GRIB_Ny:                                  93
    GRIB_cfName:                              unknown
    GRIB_cfVarName:                           tp
    GRIB_dataType:                            fc
    ...                                       ...
    description:                              Parameters of the genextreme di...
    method:                                   ML
    estimator:                                Maximum likelihood
    scipy_dist:                               genextreme
    units:                                    
    history:                                  [2024-08-08 14:24:26] fit: Esti...

In [7]:
params_ns.t2m_nanmean.values

array([[nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan]])

In [8]:
# params_ns.tp_nansum.values